#  Install package

In [409]:
# ! pip install pyvis
# ! pip install networkx
# ! pip install openai
# ! pip install contextily
# ! pip install matplotlib_scalebar
# ! pip install langchain
# ! pip install langchain-openai
# ! pip install geojson
# ! pip install OSMPythonTools

# Import package

In [410]:
%load_ext autoreload
%autoreload 2
    
import os
import requests
import networkx as nx
import pandas as pd
import geopandas as gpd
# from pyvis.network import Network
from openai import OpenAI
from IPython.display import display, HTML, Code
from IPython.display import clear_output

import base64

import pickle
import sys
import osmnx as ox



import LLM_Find_Constants as constants
import helper

from LLM_Find_kernel import Solution

from langchain_openai import ChatOpenAI

from langchain_core.prompts import ChatPromptTemplate





The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [411]:
OpenAI_key = helper.load_OpenAI_key()

## Input task and data desciption

In [412]:
# isReview = True
isReview = False

In [413]:
# # Case 1
# task_name ='OSM_PA_boundary'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\OSM_PA_boundary.gpkg.'
# task = rf'''1. Download the administrative boundary of Pennsylvania State, USA from OpenStreetMap.
# 2. Save the downloaded data in GeoPackage format, save it at: {saved_fname} 
# '''

# task_name ='OSM_SC_boundary'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\OSM_SC_boundary.gpkg.'
# task = rf'''1. Download the administrative boundary of South Carolina State, USA from OpenStreetMap.
# 2. Save the downloaded data in GeoPackage format, save it at: {saved_fname} 
# '''

# task_name ='OSM_PA_hospital'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\OSM_PA_hospital.gpkg.'
# task = rf'''1. Download all hospitals in Pennsylvania State, USA, from OpenStreetMap.
# 2. Save the downloaded data as points in GeoPackage format, save it at: {saved_fname} 
# '''

# task_name ='OSM_SC_hospital'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\OSM_SC_hospital.gpkg.'
# task = rf'''1. Download all hospitals in South Carolina State, USA, from OpenStreetMap.
# 2. Save the downloaded data as points in GeoPackage format, save it at: {saved_fname} 
# '''

# task_name ='OSM_SC_school'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\OSM_SC_school.gpkg.'
# task = rf'''1. Download all schools in South Carolina State, USA, from OpenStreetMap.
# 2. Save the downloaded data as points in GeoPackage format, save it at: {saved_fname} 
# '''

# task_name ='OSM_CA_park'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\OSM_CA_parks.gpkg.'
# task = rf'''1. Download all parks in California, USA, from OpenStreetMap, including urban public, recreation, state, and national parks.
# 2. Save the downloaded data as points in GeoPackage format, save it at: {saved_fname} 
# # '''

# task_name ='OSM_USA_university'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\OSM_USA_universities.gpkg.'
# task = rf'''1. Download all universities and colleges in USA from OpenStreetMap.
# 2. Save the downloaded data as points in GeoPackage format, save it at: {saved_fname} 
# '''

# task_name ='OSM_state_college_street'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\OSM_state_college_street.gpkg.'
# task = rf'''1. Download all streets of State College, Pennsylvania, USA, from OpenStreetMap.
# 2. Save the downloaded data as polylines in GeoPackage format, save it at: {saved_fname} 
# '''

# task_name ='Census_SC_tract'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\Census_SC_tract.gpkg.'
# if os.path.exists(saved_fname):
#     os.remove(saved_fname)
# task = rf'''1. Download all census tract boundaries in South Carolina, USA.
# 2. Save the downloaded data as polygons in GeoPackage format, save it at: {saved_fname} 
# '''

# task_name ='Census_SC_blockgroups'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\Census_SC_blockgroups.gpkg.'
# if os.path.exists(saved_fname):
#     os.remove(saved_fname)
# task = rf'''1. Download all Census block group boundaries in South Carolina, USA.
# 2. Save the downloaded data as polygons in GeoPackage format, save it at: {saved_fname} 
# '''

# task_name ='OSM_Nigeria_boundary'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\OSM_Nigeria_boundary.gpkg.'
# task = rf'''1. Download the administrative boundary of Nigeria from OpenStreetMap.
# 2. Save the downloaded data in GeoPackage format, save it at: {saved_fname} 
# '''


# task_name ='OSM_Afghanistan_boundary'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\OSM_Afghanistan_boundary.gpkg.'
# task = rf'''1. Download the administrative boundary of Afghanistan from OpenStreetMap.
# 2. Save the downloaded data in GeoPackage format, save it at: {saved_fname} 
# '''

# Case 1
# task_name ='Census_Centre_boundary'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\Census_Centre_boundary.gpkg.'
# task = rf'''1. Download the administrative boundary of Centre County of Pennsylvania State, USA from Census Bureau.
# 2. Save the downloaded data in GeoPackage format, save it at: {saved_fname} 
# '''

# task_name ='Census_SC_countries_boundary'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\Census_SC_counties_boundary.gpkg.'
# task = rf'''1. Download the administrative boundary of all Counties of South Carolina from Census Bureau.
# 2. Save the downloaded data in GeoPackage format, save it at: {saved_fname} 
# '''

# # This is a difficult case, the above cases were okay during the test.
# task_name ='Wuhan_railway_network'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\Wuhan_Railway_network.gpkg.'
# task = rf'''1. Download the railway network in Wuhan, Hubei, China.
# 2. Save the downloaded data as polylines in GeoPackage format, save it at: {saved_fname} 
# '''
#  Using "Hubei Province" may not return polygons.


# task_name ='OSM_Nigeria_railway'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\OSM_Nigeria_railway.gpkg.'
# task = rf'''1. Download the railway network of Nigeria from OpenStreetMap.
# 2. Save the downloaded data in GeoPackage format, save it at: {saved_fname} 
# '''

# task_name ='Census_SC_counties_population'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\Census_SC_counties_population.csv'
# task = rf'''1. Download latest population for each county in South Carolina.
# 2. Save the downloaded data as CSV files, save it at: {saved_fname} 
# '''

# task_name ='Census_SC_Richland_race_population'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\Census_SC_Richland_race_population.csv'
# task = rf'''1. Download latest population by race for Richland county in South Carolina.
# 2. Save the downloaded data as CSV files, save it at: {saved_fname} 
# '''

# task_name ='Census_PA_counties_race_population'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\Census_PA_counties_race_population.csv'
# task = rf'''1. Download latest population by race for all counties in Pennsylvania.
# 2. Save the downloaded data as CSV files, save it at: {saved_fname} 
# '''


# task_name ='Census_US_states_population'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\Census_US_states_population.csv'
# task = rf'''1. Download latest population for all states in USA.   
# 2. Save the downloaded data as CSV files, save it at: {saved_fname} 
# '''

# task_name ='Census_US_states_senior_population'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\Census_US_states_senior_population.csv'
# task = rf'''1. Download the latest senior population for all states in the USA.   
# 2. Save the downloaded data as CSV files, save it at: {saved_fname} 
# '''
 

# task_name ='Census_US_states_education_population'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\Census_US_states_education_population.csv'
# task = rf'''1. Download latest population by higher education attainment over 25 for all states in USA, both male and female, together with the entire population of each state.   
# 2. Save the downloaded data as CSV files, save it at: {saved_fname} 
# '''

task_name ='Census_US_county_household_income'
saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\Census_US_county_household_income.csv'
task = rf'''1. Download the latest median household income data for each county in the USA.   
2. Save the downloaded data as CSV files, save it at: {saved_fname} 
'''


if os.path.exists(saved_fname):
    os.remove(saved_fname)

save_dir = os.path.join(os.getcwd(), task_name)
os.makedirs(save_dir, exist_ok=True)
# 
# model=r"gpt-4"
# model = r'gpt-4-turbo-preview'
# model_name = "gpt-3.5-turbo"


model_name = r'gpt-4o'
# model_name = r'gpt-4'

model = ChatOpenAI(api_key=OpenAI_key, model=model_name, temperature=1)

In [414]:
%load_ext autoreload
%autoreload 2
import helper
    
source_select_prompt_str = helper.create_select_prompt(task=task)

print(source_select_prompt_str)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Your role: A professional Python programmer in geographic information science (GIScience). You have worked on GIScience for more than 20 years and know every detail and pitfall when collecting data and coding. You know which websites you can get suitable spatial data and know the methods or tricks to download data, such as OpenStreetMap, Census Bureau, or various APIs. You are also experienced in processing the downloaded data, including saving them in suitable formats, map projections, and creating detailed and useful meta-data.
 
Your mission: select a suitable data source from the given list to download the requested geo-spatial data for this task: 1. Download the latest median household income data for each county in the USA.   
2. Save the downloaded data as CSV files, save it at: E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\Census_US_county

In [415]:
from IPython.display import clear_output

chunks = []

async for chunk in model.astream(source_select_prompt_str):
    chunks.append(chunk)
    print(chunk.content, end="", flush=True)

# clear_output(wait=True)
clear_output(wait=False)
LLM_reply_str = helper.convert_chunks_to_str(chunks=chunks)

print("Select the data source: \n")
print(LLM_reply_str)

Select the data source: 

{'Explaination': "Since the task requests median household income data for each county in the USA, I should download the data from the US Census Bureau demography.", "Selected data source": 'US Census Bureau demography'}


In [416]:
import ast
select_source = ast.literal_eval(LLM_reply_str)

selected_data_source= select_source['Selected data source']
data_source_ID = constants.data_source_dict[selected_data_source]['ID']

print("selected_data_source:", selected_data_source)
print("data_source_ID:", data_source_ID)

handbook_list = constants.handbooks[f"{data_source_ID}"]
handbook_str =  '\n'.join([f"{idx + 1}. {line}" for idx, line in enumerate(handbook_list)])
print()
print(f"Handbook:\n{handbook_str}")

selected_data_source: US Census Bureau demography
data_source_ID: US_Census_demography

Handbook:
1. If you need an API key, you can use this: b12026d61228a4b0d441ae7aa93f1ea222877503
2. Prefer the office APIs, do not use other Python pacakges such as `census`.
3. Store the returns into CSV files, please make sure using descriptive headers without special characters. The header should come from the `label` value in the variable descriptions in https://api.census.gov/data/2021/acs/acs5/variables.json. Note that you may need to change the year accordingly. You need to download this JSON file and read the variable labels from it.
4. Add the year of the data as a column to the saved CSV files.
5. Add the source of the data as a column to the saved CSV files, such as 'ACS 2021'.
6. Put your reply into a Python code block. Explanation or conversation can be Python comments at the begining of the code block(enclosed by ```python and ```).
7. The download code is only in a function named 'down

In [417]:
%load_ext autoreload
%autoreload 2

download_prompt_str = helper.create_download_prompt(task, selected_data_source, handbook_str)

print(download_prompt_str)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Your role: A professional Python programmer in geographic information science (GIScience). You have worked on GIScience for more than 20 years and know every detail and pitfall when collecting data and coding. You know which websites you can get suitable spatial data and know the methods or tricks to download data, such as OpenStreetMap, Census Bureau, or various APIs. You are also experienced in processing the downloaded data, including saving them in suitable formats, map projections, and creating detailed and useful meta-data. When downloading geo-spatial data, the handbook for a perticular data source is provided, you can follow it, and write Python code carefully to download the data. 
 
Your mission: download geo-spatial data from the given data source for this task: 1. Download the latest median household income data for each county in the USA.   
2. Save the downloaded data as CSV files, sav

In [418]:
from IPython.display import clear_output

chunks = []

async for chunk in model.astream(download_prompt_str):
    chunks.append(chunk)
    print(chunk.content, end="", flush=True)

# clear_output(wait=True)
clear_output(wait=False)
LLM_reply_str = helper.convert_chunks_to_str(chunks=chunks)
print(LLM_reply_str)

```python
import requests
import pandas as pd

def download_data():
    # Define API endpoint and parameters
    base_url = "https://api.census.gov/data/2021/acs/acs5"
    api_key = "b12026d61228a4b0d441ae7aa93f1ea222877503"
    # B19013_001E:Estimate!!Median household income in the past 12 months (in 2021 inflation-adjusted dollars)
    variables = ["NAME", "GEO_ID", "B19013_001E"]
    params = {
        "get": ",".join(variables),
        "for": "county:*",
        "key": api_key
    }

    # Make the API request
    response = requests.get(base_url, params=params)
    if response.status_code != 200:
        raise RuntimeError(f"Failed to fetch data: {response.text}")

    # Parse the JSON response
    data = response.json()

    # Extract headers based on available information and required fields
    headers = ["Name", "Geo ID", "Median Household Income", "State Code", "County Code"]
    
    # Convert to DataFrame for easier processing
    df = pd.DataFrame(data[1:], columns=data[0

In [419]:
code = helper.extract_code_from_str(LLM_reply_str, task)
display(Code(code, language='python'))

import requests
import pandas as pd

def download_data():
    # Define API endpoint and parameters
    base_url = "https://api.census.gov/data/2021/acs/acs5"
    api_key = "b12026d61228a4b0d441ae7aa93f1ea222877503"
    # B19013_001E:Estimate!!Median household income in the past 12 months (in 2021 inflation-adjusted dollars)
    variables = ["NAME", "GEO_ID", "B19013_001E"]
    params = {
        "get": ",".join(variables),
        "for": "county:*",
        "key": api_key
    }

    # Make the API request
    response = requests.get(base_url, params=params)
    if response.status_code != 200:
        raise RuntimeError(f"Failed to fetch data: {response.text}")

    # Parse the JSON response
    data = response.json()

    # Extract headers based on available information and required fields
    headers = ["Name", "Geo ID", "Median Household Income", "State Code", "County Code"]
    
    # Convert to DataFrame for easier processing
    df = pd.DataFrame(data[1:], columns=data[0])

    # 

import requests
import pandas as pd

def download_data():
    # Define API endpoint and parameters
    base_url = "https://api.census.gov/data/2021/acs/acs5"
    api_key = "b12026d61228a4b0d441ae7aa93f1ea222877503"
    # B19013_001E:Estimate!!Median household income in the past 12 months (in 2021 inflation-adjusted dollars)
    variables = ["NAME", "GEO_ID", "B19013_001E"]
    params = {
        "get": ",".join(variables),
        "for": "county:*",
        "key": api_key
    }

    # Make the API request
    response = requests.get(base_url, params=params)
    if response.status_code != 200:
        raise RuntimeError(f"Failed to fetch data: {response.text}")

    # Parse the JSON response
    data = response.json()

    # Extract headers based on available information and required fields
    headers = ["Name", "Geo ID", "Median Household Income", "State Code", "County Code"]
    
    # Convert to DataFrame for easier processing
    df = pd.DataFrame(data[1:], columns=data[0])

    # Rename columns based on the headers we defined
    df.columns = headers
    
    # Adding additional required columns
    df['Year'] = 2021
    df['Source'] = 'ACS 2021'
    
    # Define output path
    output_path = "E:/OneDrive_PSU/OneDrive - The Pennsylvania State University/Research_doc/LLM-Find/Downloaded_Data/Census_US_county_household_income.csv"
    
    # Save the DataFrame to a CSV file
    df.to_csv(output_path, index=False)

# Execute the function to download data
download_data()

In [420]:

%autoreload 2
%load_ext autoreload
    
code = helper.execute_complete_program(code=code, try_cnt=10, task=task, model_name=model_name)
display(Code(code, language='python'))


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


-------------- Running code (trial # 1/10) --------------




--------------- Done ---------------




import requests
import pandas as pd

def download_data():
    # Define API endpoint and parameters
    base_url = "https://api.census.gov/data/2021/acs/acs5"
    api_key = "b12026d61228a4b0d441ae7aa93f1ea222877503"
    # B19013_001E:Estimate!!Median household income in the past 12 months (in 2021 inflation-adjusted dollars)
    variables = ["NAME", "GEO_ID", "B19013_001E"]
    params = {
        "get": ",".join(variables),
        "for": "county:*",
        "key": api_key
    }

    # Make the API request
    response = requests.get(base_url, params=params)
    if response.status_code != 200:
        raise RuntimeError(f"Failed to fetch data: {response.text}")

    # Parse the JSON response
    data = response.json()

    # Extract headers based on available information and required fields
    headers = ["Name", "Geo ID", "Median Household Income", "State Code", "County Code"]
    
    # Convert to DataFrame for easier processing
    df = pd.DataFrame(data[1:], columns=data[0])

    # Rename columns based on the headers we defined
    df.columns = headers
    
    # Adding additional required columns
    df['Year'] = 2021
    df['Source'] = 'ACS 2021'
    
    # Define output path
    output_path = "E:/OneDrive_PSU/OneDrive - The Pennsylvania State University/Research_doc/LLM-Find/Downloaded_Data/Census_US_county_household_income.csv"
    
    # Save the DataFrame to a CSV file
    df.to_csv(output_path, index=False)

# Execute the function to download data
download_data()

In [421]:
# Note: the headers are usually wrong.0

In [422]:
# data[1:]

# Show the downloaded data

In [423]:
saved_fname

'E:\\OneDrive_PSU\\OneDrive - The Pennsylvania State University\\Research_doc\\LLM-Find\\Downloaded_Data\\Census_US_county_household_income.csv'

In [424]:
ext_name = saved_fname[-4:]
ext_name

'.csv'

In [425]:
if ext_name == '.csv':
    df = pd.read_csv(saved_fname, encoding='ISO-8859-1')
    results_df = df
    # print(df)

if ext_name == 'gpkg':
    gdf = gpd.read_file(saved_fname)
    results_df = gdf
    gdf.plot()
    # print(gdf)

results_df

,Name,Geo ID,Median Household Income,State Code,County Code,Year,Source
0,"Autauga County, Alabama",0500000US01001,62660,1,1,2021,ACS 2021
1,"Baldwin County, Alabama",0500000US01003,64346,1,3,2021,ACS 2021
2,"Barbour County, Alabama",0500000US01005,36422,1,5,2021,ACS 2021
3,"Bibb County, Alabama",0500000US01007,54277,1,7,2021,ACS 2021
4,"Blount County, Alabama",0500000US01009,52830,1,9,2021,ACS 2021
...,...,...,...,...,...,...,...
3216,"Vega Baja Municipio, Puerto Rico",0500000US72145,21507,72,145,2021,ACS 2021
3217,"Vieques Municipio, Puerto Rico",0500000US72147,14942,72,147,2021,ACS 2021
3218,"Villalba Municipio, Puerto Rico",0500000US72149,20722,72,149,2021,ACS 2021
3219,"Yabucoa Municipio, Puerto Rico",0500000US72151,17267,72,151,2021,ACS 2021


In [427]:
# results_df['State'].sum()